In [1]:
from __future__ import division

import os
import time

import cv2
import numpy as np

In [2]:
LABELSPATH = '../../darknet/data/coco.names'
CONFIGPATH = '../../darknet/cfg/yolov3.cfg'
WEIGHTSPATH = '../yolov3.weights'

CONFIDENCE_THS = 0.5
NMS_THS = 0.3

# IMAGE

In [5]:
LABELS = open(LABELSPATH).read().strip().split("\n")

np.random.seed(42)
COLORS = np.random.randint(0,255,size=(len(LABELS),3),dtype="uint8")

net = cv2.dnn.readNetFromDarknet(CONFIGPATH,WEIGHTSPATH)

In [5]:
def test_on_image(net,image):
    (H,W) = image.shape[:2]

    output_ln = net.getLayerNames()
    output_ln = [output_ln[i[0]-1] for i in net.getUnconnectedOutLayers()]

    blob = cv2.dnn.blobFromImage(image,1/255.0,(416,416),swapRB=True,crop=False)
    net.setInput(blob)
    start = time.time()
    layer_outputs = net.forward(output_ln)
    end = time.time()

    print('[INFO] YOLO took {:.6f} secs'.format(end-start))

    # output storages
    boxes = []
    confidences = []
    classIDs = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions
            if confidence > CONFIDENCE_THS:
                # scale the bounding box coordinates back relative to the size of the image
                box = detection[0:4] * np.array([W,H,W,H])
                (centerX,centerY,width,height) = box.astype("int")

                # derive the top left corner of the bounding box
                x = int(centerX-(width/2))
                y = int(centerY-(height/2))

                # update our lists
                boxes.append([x,y,int(width),int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    
    idxs = cv2.dnn.NMSBoxes(boxes,confidences,CONFIDENCE_THS,NMS_THS)

    return idxs,boxes,confidences,classIDs

In [17]:
def draw_outputs(idxs,boxes,confidences,classIDs,image):
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x,y) = (boxes[i][0],boxes[i][1])
            (w,h) = (boxes[i][2],boxes[i][3])

            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(image,(x,y),(x+w,y+h),color,2)
            text = "{}:{:.4f}".format(LABELS[classIDs[i]],confidences[i])
            cv2.putText(image,text,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
    
    return image

In [21]:
imagefile = '../images/dog-cycle-car.png'
image = cv2.imread(imagefile)
idxs,boxes,confidences,classIDs = test_on_image(net,image)
output_image = draw_outputs(idxs,boxes,confidences,classIDs,image)

[INFO] YOLO took 0.245877 secs


In [22]:
cv2.imshow("",output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# CAMERA

In [3]:
LABELS = open(LABELSPATH).read().strip().split("\n")

np.random.seed(42)
COLORS = np.random.randint(0,255,size=(len(LABELS),3),dtype="uint8")

net = cv2.dnn.readNetFromDarknet(CONFIGPATH,WEIGHTSPATH)

In [10]:
cam = cv2.VideoCapture(0) #0=front-cam, 1=back-cam
cam.set(cv2.CAP_PROP_FRAME_WIDTH,1300)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT,1500)

if cam.read() == False:
    cam.open()

if not cam.isOpened():
    raise IOError("cannot open webcam")

while True:
    # start = time.time()
    ret,frame = cam.read()

    if not ret:
        raise IOError("cannot receive frame")
    
    idxs,boxes,confidences,classIDs = test_on_image(net,frame)
    
    if len(idxs) > 0 :
        for i in idxs.flatten():
            (x,y) = (boxes[i][0],boxes[i][1])
            (w,h) = (boxes[i][2],boxes[i][3])

            # draw bounding box and label on the frame
            color = [int(c) for c in COLORS[classIDs[i]]]
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],confidences[i])
            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
            frame = cv2.putText(frame,text,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)

        
    cv2.imshow('',frame)
    if cv2.waitKey(1) == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

[INFO] YOLO took 0.240918 secs
[INFO] YOLO took 0.274020 secs
[INFO] YOLO took 0.264949 secs
[INFO] YOLO took 0.255446 secs
[INFO] YOLO took 0.243302 secs
[INFO] YOLO took 0.289570 secs
[INFO] YOLO took 0.302460 secs
[INFO] YOLO took 0.302484 secs
[INFO] YOLO took 0.282288 secs
[INFO] YOLO took 0.304779 secs
[INFO] YOLO took 0.299526 secs
[INFO] YOLO took 0.219332 secs
[INFO] YOLO took 0.246967 secs
[INFO] YOLO took 0.236539 secs
[INFO] YOLO took 0.206389 secs
[INFO] YOLO took 0.235061 secs
[INFO] YOLO took 0.213789 secs
[INFO] YOLO took 0.212499 secs
[INFO] YOLO took 0.213173 secs
[INFO] YOLO took 0.239292 secs
[INFO] YOLO took 0.182205 secs
[INFO] YOLO took 0.173754 secs
[INFO] YOLO took 0.190296 secs
[INFO] YOLO took 0.185124 secs
[INFO] YOLO took 0.168090 secs
[INFO] YOLO took 0.185907 secs
[INFO] YOLO took 0.170810 secs
[INFO] YOLO took 0.168427 secs
[INFO] YOLO took 0.181617 secs
[INFO] YOLO took 0.179216 secs
[INFO] YOLO took 0.184745 secs
[INFO] YOLO took 0.175838 secs
[INFO] Y

In [9]:
cam = cv2.VideoCapture(0)
ret, frame = cam.read()
cv2.imshow('',frame)
